# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [42]:
# import libraries
'''import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger']) '''
# import re
# import numpy as np
import pandas as pd
'''from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer'''
from sqlalchemy import create_engine


In [43]:
# Constants: Path and filenames
DATAPATH = '../../data/'
MSGFILE = 'messages.csv'
CATFILE = 'categories.csv'
database_filepath = 'sqlite:///drp.db'


In [44]:
# load messages dataset
messages = pd.read_csv(DATAPATH+MSGFILE)
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [45]:
duplicates = messages.duplicated()
print(duplicates.sum())

68


In [46]:
# load categories dataset
categories = pd.read_csv(DATAPATH+CATFILE)
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [47]:
# merge datasets (inner join, as dangling tuples are useless)
df = messages.merge(categories, on='id', how='inner')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [48]:
# create a dataframe of the 36 individual category columns
categories = pd.concat([categories['id'], categories['categories'].str.split(pat=';', expand=True)], axis=1)
categories.head()

,id,0,1,2,3,4,5,6,7,8,...,26,27,28,29,30,31,32,33,34,35
0,2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,7,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,8,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,9,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,12,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [49]:
# select the first row of the categories dataframe
row = categories.iloc[0]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
#category_colnames = row[0] + row[1:].apply(lambda x: (x[:len(x)-2])).tolist()
category_colnames = ['id'] + row[1:].apply(lambda x: x[:len(x)-2]).tolist()
print(category_colnames)

['id', 'related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [50]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,7,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,8,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,9,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,12,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [51]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      26248 non-null  int64 
 1   related                 26248 non-null  object
 2   request                 26248 non-null  object
 3   offer                   26248 non-null  object
 4   aid_related             26248 non-null  object
 5   medical_help            26248 non-null  object
 6   medical_products        26248 non-null  object
 7   search_and_rescue       26248 non-null  object
 8   security                26248 non-null  object
 9   military                26248 non-null  object
 10  child_alone             26248 non-null  object
 11  water                   26248 non-null  object
 12  food                    26248 non-null  object
 13  shelter                 26248 non-null  object
 14  clothing                26248 non-null  object
 15  mo

In [52]:
# Exclude the 1st column (id) from conversion
columns = (categories.columns.tolist())[1:]

for column in columns:
    
    print(column)
    # set each value to be the last character of the string
    categories[column] = categories[column].apply(lambda x: str(x)[-1] if pd.notnull(x) else x)
    # categories[column] = categories[column].apply(lambda x: x[len(x)-1:])
    #categories[column] = categories[column].apply(lambda x: x)
    
# convert column from string to numeric
categories = categories.apply(lambda col: pd.to_numeric(col, errors='coerce') if col.dtype == 'object' else col)
categories.head()

related
request
offer
aid_related
medical_help
medical_products
search_and_rescue
security
military
child_alone
water
food
shelter
clothing
money
missing_people
refugees
death
other_aid
infrastructure_related
transport
buildings
electricity
tools
hospitals
shops
aid_centers
other_infrastructure
weather_related
floods
storm
fire
earthquake
cold
other_weather
direct_report


,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   id                      26248 non-null  int64
 1   related                 26248 non-null  int64
 2   request                 26248 non-null  int64
 3   offer                   26248 non-null  int64
 4   aid_related             26248 non-null  int64
 5   medical_help            26248 non-null  int64
 6   medical_products        26248 non-null  int64
 7   search_and_rescue       26248 non-null  int64
 8   security                26248 non-null  int64
 9   military                26248 non-null  int64
 10  child_alone             26248 non-null  int64
 11  water                   26248 non-null  int64
 12  food                    26248 non-null  int64
 13  shelter                 26248 non-null  int64
 14  clothing                26248 non-null  int64
 15  money              

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [54]:
# drop the original categories column from `df`
df.drop(labels=['categories'], axis=1, inplace=True)

df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [55]:
# concatenate the original dataframe with the new `categories` dataframe
df = messages.merge(categories, on='id', how='inner')
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [56]:
# check number of duplicates
duplicates = df[df.drop(columns=['id']).duplicated()]
print(duplicates)



          id                                            message  \
164      202  ?? port au prince ?? and food. they need gover...   
165      202  ?? port au prince ?? and food. they need gover...   
658      804  elle est vraiment malade et a besoin d'aide. u...   
659      804  elle est vraiment malade et a besoin d'aide. u...   
660      804  elle est vraiment malade et a besoin d'aide. u...   
...      ...                                                ...   
25291  29022  In a field in Jallouzai, just inside Pakistan,...   
25292  29022  In a field in Jallouzai, just inside Pakistan,...   
25378  29119  Most victims (90 per cent) show little or no s...   
25379  29119  Most victims (90 per cent) show little or no s...   
25380  29119  Most victims (90 per cent) show little or no s...   

                                                original   genre  related  \
164    p bay pap la syen ak manje. Yo bezwen ed gouve...  direct        1   
165    p bay pap la syen ak manje. Yo bez

In [57]:
# drop duplicates
df = df.drop_duplicates(subset=df.columns.difference(['id']))


In [58]:
# check number of duplicates
duplicates = df[df.drop(columns=['id']).duplicated()]
print(duplicates)


Empty DataFrame
Columns: [id, message, original, genre, related, request, offer, aid_related, medical_help, medical_products, search_and_rescue, security, military, child_alone, water, food, shelter, clothing, money, missing_people, refugees, death, other_aid, infrastructure_related, transport, buildings, electricity, tools, hospitals, shops, aid_centers, other_infrastructure, weather_related, floods, storm, fire, earthquake, cold, other_weather, direct_report]
Index: []

[0 rows x 40 columns]


In [59]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26381,30261,The training demonstrated how to enhance micro...,NaN,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26382,30262,A suitable candidate has been selected and OCH...,NaN,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26383,30263,"Proshika, operating in Cox's Bazar municipalit...",NaN,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26384,30264,"Some 2,000 women protesting against the conduc...",NaN,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
# drop 193 rows with 'related' == 2 values
# Identify the rows to drop
rows_to_drop = df[df['related'] == 2].index

# Drop the rows
df.drop(rows_to_drop, inplace=True)


In [61]:
# drop empty column 'child_alone'
df.drop(columns=['child_alone'], inplace = True)

In [62]:
df = pd.get_dummies(df, columns=['genre'])


,id,message,original,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,floods,storm,fire,earthquake,cold,other_weather,direct_report,genre_direct,genre_news,genre_social
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,True,False,False
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,True,False,False
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,True,False,False
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,True,False,False
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,True,False,False


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26027 entries, 0 to 26385
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      26027 non-null  int64 
 1   message                 26027 non-null  object
 2   original                10038 non-null  object
 3   related                 26027 non-null  int64 
 4   request                 26027 non-null  int64 
 5   offer                   26027 non-null  int64 
 6   aid_related             26027 non-null  int64 
 7   medical_help            26027 non-null  int64 
 8   medical_products        26027 non-null  int64 
 9   search_and_rescue       26027 non-null  int64 
 10  security                26027 non-null  int64 
 11  military                26027 non-null  int64 
 12  water                   26027 non-null  int64 
 13  food                    26027 non-null  int64 
 14  shelter                 26027 non-null  int64 
 15  clothin

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [63]:
engine = create_engine(database_filepath)
df.to_sql('msgcat', engine, index=False, if_exists='replace')

26027

### 8. Use this notebook to complete `process_data.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `process_data.py` in the classroom on the `Project Workspace IDE` coming later.